In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import re
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from kaggle_utils import reduce_mem_usage, move_feature

In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMN_GROUP = 'DT-M'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, COLUMN_GROUP, 'is_train', 'date']

def filter_feature(path):
    if path.count(''):
        return True
    else:
        return False

paths_train_org = sorted(glob('../feature/org_use/*_train.gz'))
paths_test_org  = sorted(glob('../feature/org_use/*_test.gz'))

df_train = parallel_load_data(paths_train_org)
df_test  = parallel_load_data(paths_test_org)

In [9]:
use_cols = [col for col in df_train.columns if col not in COLUMNS_IGNORE]

cols_V = [col for col in use_cols if col.count('V') and not col.count('C1') and not col.count('C2')]
cols_V_card_addr = [col for col in cols_V if col.count('card') and col.count('addr')]

len(cols_V_card_addr)

71

In [19]:
cols_C = [col for col in use_cols if col.count('C') and not col.count('V') and not col.count('6')]
cols_C_card = [col for col in use_cols if col.count('C') and not col.count('V') and col.count('6') and col.count('card')]

In [24]:
cols_C

['502__cnt__C11',
 '502__cnt__C13',
 '507__C1-C12__ratio',
 '507__C1-C14__diff',
 '507__C1-C14__ratio',
 '507__C1-C2__diff',
 '507__C10-C12__ratio',
 '507__C10-C14__ratio',
 '507__C11-C14__ratio__ProductCD-C',
 '507__C11-C14__ratio',
 '507__C11-C2__ratio',
 '507__C12-C2__ratio__ProductCD-C',
 '507__C13-C5__ratio',
 '507__C13-C9__diff__ProductCD-W',
 '507__C13-C9__ratio__ProductCD-W',
 '507__C14-C2__diff',
 '507__C5-C9__ratio',
 '508__C1-D15__ratio__ProductCD-W',
 '508__C1-D1__ratio',
 '508__C1-D4__ratio',
 '508__C11-D15__ratio__ProductCD-W',
 '508__C11-D15__ratio',
 '508__C11-D4__ratio',
 '508__C13-D1__ratio',
 '508__C13-D4__ratio',
 '508__C14-D15__ratio__ProductCD-W',
 '508__C2-D10__ratio__ProductCD-W',
 '508__C2-D1__ratio',
 '508__C5-D15__ratio',
 '508__C8-D14__ratio__ProductCD-C',
 '508__C9-D1__ratio__ProductCD-W',
 '508__C9-D4__ratio__ProductCD-W',
 '518__cnt__M4__ProductCD-C',
 '518__cnt__M4__ProductCD-W']

In [22]:
df_corr_C = df_train[cols_C].corr()
for col in df_corr_C.columns:
    tmp = df_corr_C[col]
    print('* ', col)
    display(tmp.sort_values(ascending=False).head(5))

*  502__cnt__C11


502__cnt__C11                1.000000
502__cnt__C13                0.481502
507__C14-C2__diff            0.102644
507__C1-C2__diff             0.075124
518__cnt__M4__ProductCD-C    0.011661
Name: 502__cnt__C11, dtype: float64

*  502__cnt__C13


502__cnt__C13                      1.000000
502__cnt__C11                      0.481502
507__C12-C2__ratio__ProductCD-C    0.167128
507__C14-C2__diff                  0.055854
507__C1-C2__diff                   0.037977
Name: 502__cnt__C13, dtype: float64

*  507__C1-C12__ratio


...

508__C9-D4__ratio__ProductCD-W      1.000000
508__C1-D4__ratio                   0.989221
508__C11-D4__ratio                  0.985811
508__C13-D4__ratio                  0.955502
508__C14-D15__ratio__ProductCD-W    0.676156
Name: 508__C9-D4__ratio__ProductCD-W, dtype: float64

*  518__cnt__M4__ProductCD-C


518__cnt__M4__ProductCD-C          1.000000
507__C5-C9__ratio                  0.202853
507__C11-C2__ratio                 0.180244
507__C1-C2__diff                   0.177082
507__C12-C2__ratio__ProductCD-C    0.163470
Name: 518__cnt__M4__ProductCD-C, dtype: float64

*  518__cnt__M4__ProductCD-W


518__cnt__M4__ProductCD-W         1.000000
507__C5-C9__ratio                 0.313329
507__C13-C9__diff__ProductCD-W    0.086095
507__C11-C2__ratio                0.075907
518__cnt__M4__ProductCD-C         0.067733
Name: 518__cnt__M4__ProductCD-W, dtype: float64

In [23]:
df_corr_C_card = df_train[cols_C_card].corr()
for col in df_corr_C_card.columns:
    tmp = df_corr_C_card[col]
    print('* ', col)
    display(tmp.sort_values(ascending=False).head(5))

*  603__card1-addr1_C10__ProductCD-H_std


603__card1-addr1_C10__ProductCD-H_std    1.000000
603__card1-addr1_C4__ProductCD-H_std     0.953989
603__card1-addr1_C2__ProductCD-H_std     0.949338
603__card1-addr1_C14__ProductCD-H_std    0.907351
603__card1-addr1_C10__ProductCD-W_std    0.694323
Name: 603__card1-addr1_C10__ProductCD-H_std, dtype: float64

*  603__card1-addr1_C10__ProductCD-R_sum


603__card1-addr1_C10__ProductCD-R_sum    1.000000
603__card1-addr1_C11__ProductCD-R_sum    0.933605
603__card1-addr1_C10__ProductCD-W_sum    0.657395
603__card1_C12__ProductCD-H_sum          0.479545
603__card1_C11__ProductCD-R_sum          0.435387
Name: 603__card1-addr1_C10__ProductCD-R_sum, dtype: float64

*  603__card1-addr1_C10__ProductCD-W_std


...

603__card6-addr1_C12__ProductCD-W_sum    1.000000
603__card2-addr1_C12__ProductCD-W_sum    0.659681
603__card6-addr1_C10__ProductCD-W_sum    0.643976
603__card6-addr1_C5__ProductCD-W_std     0.380585
603__card1-addr2_C12__ProductCD-W_sum    0.379249
Name: 603__card6-addr1_C12__ProductCD-W_sum, dtype: float64

*  603__card6-addr1_C5__ProductCD-W_std


603__card6-addr1_C5__ProductCD-W_std     1.000000
603__card6-addr1_C6__ProductCD-W_std     0.834423
603__card5-addr1_C13__ProductCD-W_std    0.639936
603__card4-addr1_C11__ProductCD-W_std    0.498970
603__card2-addr1_C13__ProductCD-W_std    0.480433
Name: 603__card6-addr1_C5__ProductCD-W_std, dtype: float64

*  603__card6-addr1_C6__ProductCD-W_std


603__card6-addr1_C6__ProductCD-W_std     1.000000
603__card6-addr1_C5__ProductCD-W_std     0.834423
603__card4-addr1_C11__ProductCD-W_std    0.566950
603__card5-addr1_C13__ProductCD-W_std    0.536480
603__card2-addr1_C13__ProductCD-W_std    0.429860
Name: 603__card6-addr1_C6__ProductCD-W_std, dtype: float64

In [16]:
df_corr = df_train[cols_V_card_addr].corr()
for col in df_corr.columns:
    tmp = df_corr[col]
    print('* ', col)
    display(tmp.sort_values(ascending=False).head(5))

*  603__card1-addr1_V258__ProductCD-R_std


603__card1-addr1_V258__ProductCD-R_std    1.000000
603__card2-addr1_V70__ProductCD-R_std     0.412844
603__card5-addr1_V258__ProductCD-R_std    0.390960
603__card1-addr1_V78__ProductCD-W_std     0.389265
603__card1-addr2_V78__ProductCD-H_std     0.381740
603__card2-addr1_V70__ProductCD-H_std     0.339101
603__card1-addr2_V308__ProductCD-H_sum    0.298081
603__card2-addr2_V317__ProductCD-R_sum    0.296350
603__card1-addr2_V315__ProductCD-R_std    0.260121
603__card5-addr1_V258__ProductCD-H_std    0.258175
Name: 603__card1-addr1_V258__ProductCD-R_std, dtype: float64

*  603__card1-addr1_V308__ProductCD-H_std


603__card1-addr1_V308__ProductCD-H_std    1.000000
603__card1-addr1_V317__ProductCD-H_std    0.847482
603__card1-addr1_V308__ProductCD-H_sum    0.607684
603__card1-addr2_V308__ProductCD-H_std    0.605053
603__card2-addr1_V308__ProductCD-H_std    0.475310
603__card2-addr1_V317__ProductCD-H_std    0.455765
603__card2-addr1_V317__ProductCD-H_sum    0.355823
603__card1-addr2_V308__ProductCD-H_sum    0.345398
603__card2-addr1_V308__ProductCD-H_sum    0.343109
603__card1-addr1_V315__ProductCD-H_std    0.309274
Name: 603__card1-addr1_V308__ProductCD-H_std, dtype: float64

*  603__card1-addr1_V308__ProductCD-H_sum


...

603__card5-addr1_V315__ProductCD-S_std    1.000000
603__card4-addr1_V308__ProductCD-S_std    0.643032
603__card2-addr1_V315__ProductCD-S_std    0.499464
603__card2-addr1_V317__ProductCD-S_std    0.338170
603__card1-addr2_V70__ProductCD-S_std     0.289848
603__card2-addr1_V308__ProductCD-S_sum    0.272850
603__card1-addr1_V317__ProductCD-S_sum    0.247817
603__card1-addr2_V308__ProductCD-S_sum    0.244452
603__card1-addr2_V315__ProductCD-S_std    0.202101
603__card1-addr1_V78__ProductCD-W_std     0.102012
Name: 603__card5-addr1_V315__ProductCD-S_std, dtype: float64

*  603__card5-addr1_V317__ProductCD-H_sum


603__card5-addr1_V317__ProductCD-H_sum    1.000000
603__card5-addr1_V308__ProductCD-R_sum    0.728635
603__card2-addr1_V308__ProductCD-H_sum    0.482979
603__card4-addr1_V315__ProductCD-H_std    0.471398
603__card1-addr1_V308__ProductCD-H_sum    0.437927
603__card5-addr1_V308__ProductCD-H_std    0.388070
603__card2-addr1_V308__ProductCD-R_sum    0.371207
603__card5-addr1_V258__ProductCD-H_std    0.370228
603__card1-addr1_V70__ProductCD-W_sum     0.363956
603__card4-addr1_V317__ProductCD-H_std    0.363652
Name: 603__card5-addr1_V317__ProductCD-H_sum, dtype: float64

*  603__card6-addr1_V308__ProductCD-H_std


603__card6-addr1_V308__ProductCD-H_std    1.000000
603__card4-addr1_V317__ProductCD-H_std    0.449423
603__card2-addr1_V308__ProductCD-H_sum    0.441009
603__card2-addr1_V308__ProductCD-H_std    0.393119
603__card2-addr1_V317__ProductCD-H_sum    0.389877
603__card2-addr1_V70__ProductCD-H_std     0.381151
603__card1-addr2_V78__ProductCD-H_std     0.369156
603__card5-addr1_V258__ProductCD-H_std    0.353115
603__card5-addr1_V308__ProductCD-H_std    0.350920
603__card4-addr1_V315__ProductCD-H_std    0.346690
Name: 603__card6-addr1_V308__ProductCD-H_std, dtype: float64